In [17]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import pandas as pd
from tqdm import tqdm

In [5]:
inPath = "/shared/3/projects/benlitterer/podcastData/hostIdentification/trainTestVal2_5/500trainLabelled.csv"
df = pd.read_csv(inPath)

In [7]:
df = df.dropna(subset=["tag"])

In [9]:
df[["left", "ent", "right"]] = df[["left", "ent", "right"]].fillna("")

In [10]:
df["fullSnip"] = df["left"] + df["ent"] + df["right"]

In [11]:
deviceNum = 1
device = torch.device("cuda:" + str(deviceNum) if torch.cuda.is_available() else "cpu")

In [12]:
model = AutoModelForSeq2SeqLM.from_pretrained("google/flan-t5-small").to(device)
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")

[2024-02-08 17:37:31,148] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [18]:
outPred = []
for snip in tqdm(df["fullSnip"]):  
    prefix = "Output the full name of the podcast hosts in the following sequence of text, and if there is no host output NONE instead: "  
    inputs = tokenizer(f"{prefix} {snip}", return_tensors="pt").to(device)
    outputs = model.generate(**inputs)
    decoded = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    outPred.append(decoded) 
    #print(snip) 
    #print(decoded) 
    #print("----------------------------") 

100%|██████████| 298/298 [00:26<00:00, 11.19it/s]


In [19]:
outPred[0:10]

[['Carl Bimchawand'],
 ['Harry Potter'],
 ['Coach Bowling'],
 ['adrian kobson'],
 ['Falbani'],
 ['Andy Dieri'],
 ['Angela Hauntley'],
 ['Francis Fukuyama'],
 ['Tammy Pateand'],
 ['Nathan Katum']]

In [23]:
outCol = pd.DataFrame(outPred, columns=["FLANT5oneShot"])

In [25]:
outCol.to_csv("/shared/3/projects/benlitterer/podcastData/hostIdentification/FLANTT5/earlyExperiments/500trainTest.csv", sep=",")